In [106]:
import datetime
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, interactive, fixed, Dropdown, Output,BoundedFloatText,BoundedIntText,HBox
from IPython.core.display import display, HTML

In [107]:
df_italy_raw = pd.read_csv('https://raw.githubusercontent.com/leogalbu/Covid/master/covid19_italy_region.csv')
df_clean_raw = pd.read_csv('https://raw.githubusercontent.com/leogalbu/Covid/master/covid_19_clean_complete.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [108]:
df_clean_conf = df_clean_raw[df_clean_raw['Confirmed'] > 0]
df_clean_deaths = df_clean_raw[df_clean_raw['Deaths'] > 0]
df_clean_rec = df_clean_raw[df_clean_raw['Recovered'] > 0]

In [109]:
ALL = 'Tutti'
def unique(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [110]:
# Global trend
last_date = df_clean_raw[df_clean_raw['Date'] == max(df_clean_raw['Date'])].reset_index()

In [111]:
group = last_date.groupby('Country/Region')['Confirmed','Deaths','Recovered'].sum().reset_index()

In [130]:
#fig =go.FigureWidget(layout=go.Layout())
#def show_n_cases(number):
    #number = int(number)
    #return group.sort_values(by='Confirmed', ascending = False).head(number).style.background_gradient(cmap ='BuGn', subset=['Confirmed']).\
#background_gradient(cmap ='Reds', subset=['Deaths'])\
#.background_gradient(cmap ='Greens', subset=['Recovered'])

In [129]:
drop_state_ = Dropdown(options = unique(confirmed_df['Country/Region']), description='Stato')
plot_output_ = Output()

def filtrami_(stato):
    plot_output_.clear_output()
    with plot_output_:
        #if stato != '':
        label = ['Confirmed', 'Deaths', 'Recovered']
        colors = ['blue', 'red', 'green']
        df_list = [confirmed_df,death_df,recovered_df]
        fig = go.Figure()
    
        for i, df in enumerate(df_list):
            x = np.array(list(df.iloc[:, 15:].columns))
            y = np.sum(np.asarray(df[df['Country/Region'] == stato].iloc[:,15:]),axis=0)
            fig.add_trace(go.Scatter(x=x,y=y, mode='markers', name=label[i], line=dict(color=colors[i])))
            fig.update_layout(title='Time series of COVID19 ' +str(confirmed_df.columns[-1]) + ' - ' +str(stato), xaxis_title='Date', yaxis_title='Number', template='presentation')
        fig.show()
            
def drop_state_handler_(change):
    filtrami_(change.new)
        
drop_state_.observe(drop_state_handler_, names = 'value')
sistema_widgets = HBox([drop_state_])

display(sistema_widgets)
display(plot_output_)

Output()

In [114]:
#def country_time_series(country):
    #label = ['Confirmed', 'Deaths', 'Recovered']
    #colors = ['blue', 'red', 'green']
    #df_list = [confirmed_df,death_df,recovered_df]
    #fig = go.Figure()
    
    #for i, df in enumerate(df_list):
        #x = np.array(list(df.iloc[:, 15:].columns))
        #y = np.sum(np.asarray(df[df['Country/Region'] == country].iloc[:,15:]),axis=0)
        #fig.add_trace(go.Scatter(x=x,y=y, mode='markers', name=label[i], line=dict(color=colors[i])))
        #fig.update_layout(title='Time series of COVID19 (03/04/2020)', xaxis_title='Date', yaxis_title='Number', template='presentation')
    #fig.show()

In [115]:
#interact(country_time_series, country = 'Italy')


In [116]:
df_location = df_clean_raw.groupby(['Country/Region','Date']).sum().reset_index().sort_values('Date', ascending =False)
df_location = df_location.drop_duplicates(subset=['Country/Region'])
df_location = df_location[df_location['Confirmed'] > 0]

In [117]:
fig = go.Figure(data=go.Choropleth(
locations = df_location['Country/Region'],
locationmode ='country names',
z = df_location['Confirmed'],
colorscale = 'Blues',
marker_line_color = 'black',
marker_line_width = 0.8))

fig.update_layout(
title_text = 'Confirmed Case 03/04/2020',
geo=dict(
showframe=False,
showcoastlines=False,
projection_type = 'equirectangular'))

In [118]:
# Animation map
fig = px.choropleth(df_clean_raw,
                   locations='Country/Region',
                   locationmode='country names',
                   color = 'Confirmed',
                   hover_name = 'Country/Region',
                   animation_frame ='Date')
fig.update_layout(title_text='Confirmed Case Animation',
                 geo=dict(
                 showframe=False,
                 showcoastlines =False),
                 annotations = [dict(
                 x=0.60,
                 y=0.1,
                 xref='paper',
                 yref='paper',
                 showarrow=False)])
fig.show()

In [119]:
#ALL = 'Tutti'
#def unique(array):
    #unique = array.unique().tolist()
    #unique.sort()
    #unique.insert(0, ALL)
    #return unique'

In [127]:
drop_state = Dropdown(options = unique(df_clean_raw['Country/Region']), description='Stato')
numero = BoundedIntText(min=1, max=1000, values=5, step=1, description='Righe')
output = Output()
plot_output = Output()

def filtrami(stato, num):
    output.clear_output()
    plot_output.clear_output()
    if (stato == ALL):
        filtering = df_clean_raw.sort_values(by='Confirmed', ascending = False).head(num)
    else:
        filtering = df_clean_raw[(df_clean_raw['Country/Region'] == stato)].sort_values(by='Confirmed', ascending = False).head(num)
    with output:
        display(filtering.style.background_gradient(cmap ='BuGn', subset=['Confirmed']).\
background_gradient(cmap ='Reds', subset=['Deaths'])\
.background_gradient(cmap ='Greens', subset=['Recovered']))
    with plot_output:
        if num > 0:
            sns.barplot(x="Confirmed", y="Date", data=filtering)
            plt.show()
            
def drop_state_handler(change):
    filtrami(change.new, numero.value)
def numero_handler(change):
     filtrami(drop_state.value, change.new)
        
drop_state.observe(drop_state_handler, names = 'value')
numero.observe(numero_handler, names='value')

#drop_state.observe(dropdown_state, names='value')

sistema_widgets = HBox([drop_state, numero])

display(sistema_widgets)
#display(drop_state)
#display(numero)
display(output)
display(plot_output)


Output()

Output()

In [121]:
# Prendo l'ultima data disponibile, la + aggiornata, ho così i dati a quella data
last_date = df_italy_raw.sort_values(['Date', 'Country', 'RegionName'])
last_date = last_date[df_italy_raw.Date == last_date.Date.max()].reset_index()

C:\Users\Leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [122]:
df_italy_group = last_date.groupby('RegionName')[['HospitalizedPatients', 'TotalHospitalizedPatients', 'HomeConfinement',
                                                'CurrentPositiveCases','NewPositiveCases', 'Recovered','Deaths',
                                                'TotalPositiveCases', 'TestsPerformed']].sum().reset_index()
df_italy_group = df_italy_group.sort_values(by='TotalPositiveCases', ascending = False)
df_italy_group.style.background_gradient(cmap='plasma')

,RegionName,HospitalizedPatients,TotalHospitalizedPatients,HomeConfinement,CurrentPositiveCases,NewPositiveCases,Recovered,Deaths,TotalPositiveCases,TestsPerformed
8,Lombardia,11762,13113,12763,25876,1292,12229,7960,46065,128286
4,Emilia-Romagna,3944,4310,7549,11859,546,1663,1811,15333,60507
13,Piemonte,3341,3794,5005,8799,558,571,983,10353,32100
20,Veneto,1670,2015,6563,8578,486,1001,532,10111,120320
17,Toscana,1135,1430,3359,4789,406,216,268,5273,40724
9,Marche,986,1150,2405,3555,136,40,503,4098,12943
7,Liguria,1120,1292,1368,2660,122,634,488,3782,12069
6,Lazio,1169,1350,1529,2879,169,369,185,3433,41575
3,Campania,503,623,1517,2140,225,149,167,2456,17404
14,Puglia,645,763,1101,1864,131,69,144,2077,16554


In [123]:
fig = px.bar(df_italy_group[['RegionName', 'HospitalizedPatients']].sort_values('HospitalizedPatients', ascending=False),
            y='HospitalizedPatients', x='RegionName', color = 'RegionName', template = 'presentation', title= 'Hospitalized Patients for Region' ) 
fig.update_layout(
    xaxis = dict(
        tickangle = 30,
        title_text = "Regione",
        title_font = {"size": 20},
        title_standoff = 50),
    yaxis = dict(
        title_text = "Numero casi",
        title_standoff = 5))
fig.show()

In [124]:
fig = px.bar(df_italy_group[['RegionName', 'Deaths']].sort_values('Deaths', ascending = False),
            y='Deaths', x='RegionName', color='RegionName', template='presentation', title='Death Cases for Region')
fig.update_layout(
    xaxis = dict(
        tickangle = 30,
        title_text = "Regione",
        title_font = {"size": 20},
        title_standoff = 50),
    yaxis = dict(
        title_text = "Numero casi",
        title_standoff = 5))
fig.show()